In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# A2A Demo


In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-a2a-demo"
REGION="us-central1"

In [ ]:
import os

In [ ]:
!pip install uv

In [ ]:
!uv sync --frozen

In [ ]:
%%bash
gcloud run deploy burger-agent \
    --source remote_seller_agents/burger_agent \
    --port=8080 \
    --allow-unauthenticated \
    --min 1 \
    --region us-central1 \
    --update-env-vars GOOGLE_CLOUD_LOCATION=$REGION \
    --update-env-vars GOOGLE_CLOUD_PROJECT=$PROJECT_ID

In [ ]:
# to get the service URL programmatically
SERVICE_URL = !gcloud run services describe burger-agent \
  --platform managed \
  --region $REGION \
  --format "value(status.url)"

BURGER_AGENT_SERVICE_URL = SERVICE_URL[0]
print(BURGER_AGENT_SERVICE_URL)

os.environ["BURGER_AGENT_HOST_OVERRIDE"] = BURGER_AGENT_SERVICE_URL

In [ ]:
%%bash
gcloud run services update burger-agent --region=us-central1 --update-env-vars=HOST_OVERRIDE=$BURGER_AGENT_HOST_OVERRIDE

In [ ]:
import requests
import json
res = requests.get(f'{BURGER_AGENT_SERVICE_URL}/.well-known/agent.json')
print(json.dumps(json.loads(res.content), indent=4))

In [ ]:
%%bash
gcloud run deploy pizza-agent \
    --source remote_seller_agents/pizza_agent \
    --port=8080 \
    --allow-unauthenticated \
    --min 1 \
    --region us-central1 \
    --update-env-vars GOOGLE_CLOUD_LOCATION=$REGION \
    --update-env-vars GOOGLE_CLOUD_PROJECT=$PROJECT_ID

In [ ]:
# to get the service URL programmatically
PIZZA_AGENT_SERVICE_URL = !gcloud run services describe pizza-agent \
  --platform managed \
  --region $REGION \
  --format "value(status.url)"

PIZZA_AGENT_SERVICE_URL = PIZZA_AGENT_SERVICE_URL[0]
print(PIZZA_AGENT_SERVICE_URL)
import os
os.environ["PIZZA_AGENT_SERVICE_URL"] = PIZZA_AGENT_SERVICE_URL

In [ ]:
%%bash
gcloud run services update pizza-agent --region=us-central1 --update-env-vars=HOST_OVERRIDE=$PIZZA_AGENT_SERVICE_URL

In [ ]:
res = requests.get(f'{PIZZA_AGENT_SERVICE_URL}/.well-known/agent.json')
print(json.dumps(json.loads(res.content), indent=4))

In [ ]:
!gcloud storage buckets create gs://purchasing-concierge-$PROJECT --location=us-central1

In [ ]:
os.environ["AGENT_ENGINE_RESOURCE_NAME"] = "projects/25570882233/locations/us-central1/reasoningEngines/8048165630576164864"

In [ ]:
%%bash
curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/${AGENT_ENGINE_RESOURCE_NAME}:streamQuery?alt=sse -d '{
  "class_method": "stream_query",
  "input": {
    "user_id": "user_123",
    "message": "List available burger menu please",
  }
}'


In [ ]:
import requests
import subprocess
import os
import json

# 1. Get the AGENT_ENGINE_RESOURCE_NAME from environment variables
# Example: "projects/your-gcp-project-id/locations/us-central1/agents/your-agent-id/engines/your-engine-id"
agent_engine_resource_name = os.getenv("AGENT_ENGINE_RESOURCE_NAME")
if not agent_engine_resource_name:
    raise ValueError("The 'AGENT_ENGINE_RESOURCE_NAME' environment variable is not set.")

# 2. Construct the full URL
url = f"https://us-central1-aiplatform.googleapis.com/v1/{agent_engine_resource_name}:streamQuery"

# 3. Get the gcloud access token
try:
    token_process = subprocess.run(
        ["gcloud", "auth", "print-access-token"],
        capture_output=True,
        text=True,
        check=True
    )
    access_token = token_process.stdout.strip()
except (subprocess.CalledProcessError, FileNotFoundError) as e:
    print(f"Error getting gcloud access token: {e}")
    print("Please ensure the gcloud CLI is installed, authenticated, and in your system's PATH.")
    exit(1)

# 4. Define the request headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
}

# 5. Define the request payload (data)
payload = {
    "class_method": "stream_query",
    "input": {
        "user_id": "user_123",
        "message": "List available pizza menu please",
    }
}

# 6. Make the POST request with streaming enabled
try:
    with requests.post(
        url,
        headers=headers,
        json=payload, # requests handles JSON serialization
        params={"alt": "sse"}, # Handles the query parameter
        stream=True  # This is key for streaming responses
    ) as response:
        # Check for HTTP errors
        response.raise_for_status()
        
        print("Successfully connected to stream. Waiting for data...")
        # Iterate over the response line by line as data arrives
        stream_chunk = 0
        for line in response.iter_lines():
            if line:
                # Decode bytes to string
                decoded_line = line.decode('utf-8')
                print(f"------------------------[{str(stream_chunk)}]------------------------")
                print(json.dumps(json.loads(decoded_line), indent=4))
                stream_chunk+=1

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")